In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier 
import pickle
from sklearn.feature_selection import mutual_info_classif

In [ ]:
df=pd.read_csv('data/ISCX_training.csv')

In [ ]:
filtered_columns=list(set(my_columns)-set(['Unnamed: 0','Label','SrcAddr', 'DstAddr', 'Proto', 'Sport', 'Dport', 'State', 'StartTime', 'LastTime','SrcMac', 'DstMac']))

In [ ]:
filtered_columns

In [ ]:
print(len(filtered_columns))

In [ ]:
# # Load the ig_scores array
ctu_ig_scores = np.load('info_gain_result/ctu_ig_scores.npy')
iscx_ig_scores=np.load('info_gain_result/iscx_ig_scores.npy')
isot_ig_scores=np.load('info_gain_result/isot_ig_scores.npy')

In [ ]:

labels = ['sTos', 'dTos', 'SrcWin', 'DstWin', 'sHops', 'dHops', 'sTtl', 'dTtl', 'TcpRtt', 'SynAck', 'AckDat', 'SrcPkts',
          'DstPkts', 'SrcBytes', 'DstBytes', 'SAppBytes', 'DAppBytes', 'Dur', 'TotPkts', 'TotBytes', 'TotAppByte',
          'Rate', 'SrcRate', 'DstRate']

# Function to create a bar plot with bold labels and increased font size
def plot_ig_scores(data, title, dataset_name):
    plt.figure(figsize=(12, 6))
    plt.bar(range(len(data)), data)
    plt.xlabel('Features', fontsize=14, fontweight='bold')
    plt.ylabel('Information Gain', fontsize=14, fontweight='bold')
    plt.title(title + ' - ' + dataset_name, fontsize=14, fontweight='bold')
    plt.xticks(range(len(labels)), labels, rotation=45, fontsize=10, fontweight='bold')
    plt.yticks(fontsize=12, fontweight='bold')
    plt.show()

# Plotting for each dataset
plot_ig_scores(iscx_ig_scores, 'Information Gain Scores', 'ISCX Dataset')
plot_ig_scores(isot_ig_scores, 'Information Gain Scores', 'ISOT Dataset')

# Calculate average information gain and plot
avg_ig_scores = np.mean([ctu_ig_scores, iscx_ig_scores, isot_ig_scores], axis=0)
plot_ig_scores(avg_ig_scores, 'Average Information Gain Scores', 'Average')

In [ ]:


# Assuming you have ctu_ig_scores, iscx_ig_scores, and isot_ig_scores as arrays
labels = ['sTos', 'dTos', 'SrcWin', 'DstWin', 'sHops', 'dHops', 'sTtl', 'dTtl', 'TcpRtt', 'SynAck', 'AckDat', 'SrcPkts',
          'DstPkts', 'SrcBytes', 'DstBytes', 'SAppBytes', 'DAppBytes', 'Dur', 'TotPkts', 'TotBytes', 'TotAppByte',
          'Rate', 'SrcRate', 'DstRate']



# Bar plot for iscx_ig_scores
plt.figure(figsize=(12, 6))
plt.bar(range(len(iscx_ig_scores)), iscx_ig_scores)
plt.xlabel('Features')
plt.ylabel('Information Gain')
plt.title('Information Gain Scores - ISCX Dataset')
plt.xticks(range(len(labels)), labels, rotation=45)
plt.show()

# Bar plot for isot_ig_scores
plt.figure(figsize=(12, 6))
plt.bar(range(len(isot_ig_scores)), isot_ig_scores)
plt.xlabel('Features')
plt.ylabel('Information Gain')
plt.title('Information Gain Scores - ISOT Dataset')
plt.xticks(range(len(labels)), labels, rotation=45)
plt.show()

# Average information gain and bar plot
avg_ig_scores = np.mean([iscx_ig_scores, isot_ig_scores], axis=0)
plt.figure(figsize=(12, 6))
plt.bar(range(len(avg_ig_scores)), avg_ig_scores)
plt.xlabel('Features')
plt.ylabel('Average Information Gain')
plt.title('Average Information Gain Scores')
plt.xticks(range(len(labels)), labels, rotation=45)
plt.show()


In [ ]:
potential_thresholds = np.linspace(0, max(avg_ig_scores), num=20)
print(potential_thresholds)

In [ ]:
def find_best_threshold(filename):

    avg_ig_scores=np.array([0.00040332, 0.00161157, 0.12017481, 0.04676683, 0.14954481,\
        0.05875154, 0.15869463, 0.113153  , 0.04962509, 0.04982147,\
        0.04508607, 0.08877224, 0.07776499, 0.13168685, 0.0932552 ,\
        0.01016336, 0.00898032, 0.12269696, 0.08166047, 0.13271621,\
        0.01046889, 0.11122463, 0.08523292, 0.0630542 ])
    potential_thresholds = np.linspace(0, max(avg_ig_scores), num=20)
    data=pd.read_csv(r"data/isot_botnet.csv")
    data_sel=data[['sTos', 'dTos', 'SrcWin', 'DstWin', 'sHops', 'dHops', 'sTtl', 'dTtl', 'TcpRtt', 'SynAck', 'AckDat', 'SrcPkts',\
    'DstPkts', 'SrcBytes', 'DstBytes', 'SAppBytes', 'DAppBytes', 'Dur', 'TotPkts', 'TotBytes', 'TotAppByte',\
    'Rate', 'SrcRate', 'DstRate']]
    X_train=data_sel.values
    y_train=data['Label'].tolist()
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.99, random_state=42)
    best_threshold = None
    best_score = -np.inf  # assuming a higher score is better, adjust if necessary
    threshold_scores=[]
    for threshold in potential_thresholds:
        print(threshold)
        # select features with IG above the threshold
        selected_features = np.where(avg_ig_scores >= threshold)[0]
        X_train_selected = X_train[:, selected_features]
        
        # train model using selected features
        model = RandomForestClassifier()  # replace with your model
        model.fit(X_train_selected, y_train)

        # perform cross-validation on the validation set
        cv_score = cross_val_score(model, X_train_selected, y_train, cv=2, scoring='f1').mean()
        print("This is the score we got",cv_score)
        threshold_scores.append((threshold,cv_score ))
        # update best_threshold and best_score if this model is better
        if cv_score > best_score:
            best_threshold = threshold
            best_score = cv_score
    # Save the list to a file
    with open('feature_sel_result\/'+filename+'_sel_result.pickle', "wb") as file:
        pickle.dump(threshold_scores, file)
    print(f'Best threshold: {best_threshold}, F1 Score: {best_score}')
    return threshold_scores


In [ ]:
def get_mutual_info(filename):
    data=pd.read_csv(filename,index_col=[0])
    X=data[['sTos', 'dTos', 'SrcWin', 'DstWin', 'sHops', 'dHops', 'sTtl', 'dTtl',
        'TcpRtt', 'SynAck', 'AckDat', 'SrcPkts', 'DstPkts', 'SrcBytes',
        'DstBytes', 'SAppBytes', 'DAppBytes', 'Dur', 'TotPkts', 'TotBytes',
        'TotAppByte', 'Rate', 'SrcRate', 'DstRate']]
    #ctu = ctu.drop("Unnamed: 0", axis=1)
    #X = iscx.drop(['SrcAddr', 'DstAddr','Proto','Sport','Dport','State','StartTime','LastTime','Label'], axis=1) # Drop the target column
    y = data['Label']
    ig_scores_data = mutual_info_classif(X , y)
    np.save('info_gain_result/iscx_ig_scores.npy',ig_scores_data)


In [ ]:
iscx_thresholds=find_best_threshold('path_to your data')
isot_thresholds=find_best_threshold('path to your data')

In [ ]:
isot_scores=get_mutual_info('path to your data')
iscx_scores=get_mutual_info('path to your data')

In [ ]:


# Create the plot
plt.figure(figsize=[10, 8])
plt.plot(iscx_thresholds, iscx_scores, label='ISCX')
plt.plot(isot_thresholds, isot_scores, label='ISOT')

# Annotations with increased font size and bold font
for i in range(len(iscx_scores)):
    plt.annotate(round(iscx_scores[i], 2), (iscx_thresholds[i], iscx_scores[i]), fontsize=10, fontweight='bold')
for i in range(len(isot_scores)):
    plt.annotate(round(isot_scores[i], 2), (isot_thresholds[i], isot_scores[i]), fontsize=10, fontweight='bold')

# Setting font size and font weight for labels and title
plt.xlabel('Threshold', fontsize=14, fontweight='bold')
plt.ylabel('F1 Score', fontsize=14, fontweight='bold')
plt.title('F1 Scores vs Thresholds', fontsize=16, fontweight='bold')
plt.xticks(fontsize=12, fontweight='bold')
plt.yticks(fontsize=12, fontweight='bold')

# Enhancing the legend and grid
plt.legend(fontsize=12, title_fontsize=12)
plt.grid(True)

# Display the plot
plt.show()


In [ ]:
selected_features_indices = np.where(avg_ig_scores >= 0.09)[0]

In [ ]:
selected_features = [labels[i] for i in selected_features_indices]

print(selected_features)